In [21]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from sentence_transformers import SentenceTransformer
np.random.seed(42)

In [22]:
class DatasetCFG:
    data_root = 'ml-1m'
    user_chosen_num = 1000
    num_negatives = 4

### 获取训练数据


In [23]:
# def data_preprocess():
#     ratings_path = os.path.join(DatasetCFG.data_root, 'ratings.csv')
#     movies_path = os.path.join(DatasetCFG.data_root, 'movies.csv')
#     ratings_data = pd.read_csv(ratings_path)
#     movies_data = pd.read_csv(movies_path)

#     # random_user_ids=np.random.choice(ratings_data['userId'].unique(),
#     #                             size=int(len(ratings_data['userId'].unique())*0.001),
#     #                             replace=False)
#     # ratings_data=ratings_data[ratings_data['userId'].isin(random_user_ids)]

#     ratings_data = ratings_data.merge(movies_data, on='movieId')
#     ratings_data['rank_latest'] = ratings_data.groupby(
#         ['userId'])['timestamp'].rank(method='first', ascending=True)

#     ratings_data = ratings_data.sort_values(['userId', 'rank_latest'], ascending=[
#                                             True, False]).reset_index(drop=True)

#     for _, group in tqdm(ratings_data.groupby('userId'), total=len(ratings_data['userId'].unique())):
#         user_behavior_list = []
#         for _, row in group.iterrows():
#             user_behavior = " ".join(
#                 map(str, group[group['rank_latest'] < row['rank_latest']]['movieId'].to_list()))
#             if user_behavior == '':
#                 user_behavior = ' '
#             user_behavior_list.append(user_behavior)
#         ratings_data.loc[group.index, 'user_behavior'] = user_behavior_list

#     ratings_data.to_csv(os.path.join(DatasetCFG.data_root,
#                         'ratings_data_process_1.csv'), index=False)


# def data_process_with_genome():
#     ratings_data = pd.read_csv(os.path.join(
#         DatasetCFG.data_root, 'ratings_data_process_1.csv'))
#     genome_scores_data = pd.read_csv(os.path.join(
#         DatasetCFG.data_root, 'genome-scores.csv'))
#     genome_tags_data = pd.read_csv(os.path.join(
#         DatasetCFG.data_root, 'genome-tags.csv'))

#     genome_scores_data = genome_scores_data.merge(genome_tags_data, on='tagId')
#     genome_scores_data = genome_scores_data.pivot(
#         index='movieId', columns='tag', values='relevance')
#     genome_scores_data = genome_scores_data.reset_index()

#     ratings_data = ratings_data.merge(genome_scores_data, on='movieId')
#     ratings_data.to_csv(os.path.join(DatasetCFG.data_root,
#                         'ratings_data_process_2.csv'), index=False)

In [24]:
# data_preprocess()
movies = pd.read_csv(os.path.join(DatasetCFG.data_root, "movies.dat"), sep="::", names=[
    "movieId", "title", "genres"], engine="python", encoding="ISO-8859-1")
ratings = pd.read_csv(os.path.join(DatasetCFG.data_root, "ratings.dat"), sep="::", names=[
    "userId", "movieId", "rating", "timestamp"], engine="python", encoding="ISO-8859-1")
users = pd.read_csv(os.path.join(DatasetCFG.data_root, "users.dat"), sep="::", names=[
    "userId", "gender", "age", "occupation", "zipCode"], engine="python", encoding="ISO-8859-1")

df = movies.merge(ratings, on='movieId')
df = df.merge(users, on='userId')
df.head()
df.to_csv(os.path.join(DatasetCFG.data_root,
                       'ratings_data_process.csv'), index=False)